In [ ]:
# Import the required packages
from ema_workbench import (Model,MultiprocessingEvaluator,  SequentialEvaluator, Policy, Scenario, 
                           save_results, load_results, 
                           RealParameter, IntegerParameter, CategoricalParameter, ScalarOutcome, TimeSeriesOutcome)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.analysis import (prim, 
                                    dimensional_stacking, 
                                    regional_sa, 
                                    feature_scoring, 
                                    pairs_plotting)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.analysis import prim
from ema_workbench.analysis import dimensional_stacking
from ema_workbench.analysis import feature_scoring
import numpy as np
import time
import altair as alt
import seaborn as sns
import pandas as pd
import networkx as nx
import random
import gzip, pickle, pickletools
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid') 
from numpy.lib import recfunctions as rf

ema_logging.log_to_stderr(ema_logging.INFO)

# Model Definition

The model is defined in the Model.py file and is imported here. Below the input data is loaded and the model settings for the experimentation are specified. To use the model in the EMA workbench, the model uncertainties, levers and outcomes need to be specified. 

In [ ]:
# import data
df_demand = pd.read_csv('df_demand.csv')
df_production = pd.read_csv('df_production.csv')
yhat_pred = pd.read_csv('yhat_pred.csv')
yhat_pred = list(yhat_pred.iloc[:,0])

green_list_year = np.array(df_production['renewable_generation_kWh'])  # green production in kWh for each t
fossil_list_year = np.array(df_production['fossil_generation_kWh'])

year_dem = df_demand.iloc[:35019,:] # almost a year
year_prod = df_production.iloc[:35011,:]
green_prediction = yhat_pred[:35020]  

demand_list = np.array(year_dem['E1A_kWh'])
shift_list = np.array(year_dem['shiftable_%']) / 1.5  # heating  #about 40 % perc # only shift
semishift_list = (np.array(year_dem['semi_shiftable_%']) + np.array(year_dem['shiftable_%'])) / 1.5 # about 50 % # shift en semi 
green_list = np.array(year_prod['renewable_generation_kWh'])  # green production in kWh for each t
fossil_list = np.array(year_prod['fossil_generation_kWh']) # green production in kWh for each t

In [ ]:
# Definitions of the interventions
# Base case A
p0 = {'RTP_policy':'None','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}
# Base case B
p1 = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}

# Additional separate interventions 
# Cost comparison and reduction  
p2a = {'RTP_policy':'RTP_Mismatch', 'cost_comparison':1, 'reduction': 10, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'} 
p2b = {'RTP_policy':'RTP_Mismatch','cost_comparison':1, 'reduction': 30, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}
p2c = {'RTP_policy':'RTP_Mismatch','cost_comparison':1, 'reduction': 50, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}

# Environmental friendliness comparison
p3 = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 1,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'} 

# Information campaigns separate and combined
p4a = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1.1,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}
p4b = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 0.95, 'info_campaign_sm': 3,'init_package':'value_based'}
p4c = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1.1,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 0.95, 'info_campaign_sm': 3,'init_package':'value_based'} 
p4d = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1.2,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 0.90, 'info_campaign_sm': 3,'init_package':'value_based'} 
p4e = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1.05,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 0.975, 'info_campaign_sm': 3,'init_package':'value_based'} 

# Default shifting
p5 = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'default_shift'} 

# Combinations of interventions
# Reduction and environmental comparison
p6 = {'RTP_policy':'RTP_Mismatch','cost_comparison':1, 'reduction': 30, 'env_score': 1,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}
# Environmental comparison and information campaigns
p7 = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 1,"effect_info_campaign_environment":1.1,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 0.95, 'info_campaign_sm': 3,'init_package':'value_based'}
# Reduction + info sceptics
p8 = {'RTP_policy':'RTP_Mismatch','cost_comparison':1, 'reduction': 30, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 0.95, 'info_campaign_sm': 3,'init_package':'value_based'}
# Reduction + info green 
p9 = {'RTP_policy':'RTP_Mismatch','cost_comparison':1, 'reduction': 30, 'env_score': 0,"effect_info_campaign_environment":1.1,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}
# Environmental comparison + green campaign 
p10 = {'RTP_policy':'RTP_Mismatch','cost_comparison':0, 'reduction': 0, 'env_score': 1,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}
# Flat pricing + high reduction
p11 = {'RTP_policy':'None','cost_comparison':1, 'reduction': 50, 'env_score': 0,"effect_info_campaign_environment":1,
      'info_campaign_env': 0, "effect_info_campaign_smart_meter": 1, 'info_campaign_sm': 0,'init_package':'value_based'}
# All policies except default shifting
p12 = {'RTP_policy':'RTP_Mismatch','cost_comparison':1, 'reduction': 30, 'env_score': 1,"effect_info_campaign_environment":1.1,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 0.95, 'info_campaign_sm': 3,'init_package':'value_based'}
# All policies
p13 = {'RTP_policy':'RTP_Mismatch','cost_comparison':1, 'reduction': 30, 'env_score': 1,"effect_info_campaign_environment":1.1,
      'info_campaign_env': 3, "effect_info_campaign_smart_meter": 0.95, 'info_campaign_sm': 3,'init_package':'default_shift'} 

# choose one intervention for the experiments
p=p0

In [ ]:
# import the model
from Model import *
seed1 = 44
np.random.seed(seed1)

#define function with default parameter values
def grid_model(green_list_year=green_list_year, fossil_list_year=fossil_list_year, 
               green_list=green_list, fossil_list=fossil_list, semishift_list=semishift_list, 
               shift_list=shift_list, demand_list=demand_list, green_prediction=green_prediction,
               price_flat=0.216, alpha=0.1, beta=0, gamma=0.1, RTP_policy=p['RTP_policy'],
               switch_period=int(int(len(green_list)-1) / 4), threshold_shift=0.3,
               threshold_semishift=0.9, social_pressure_effect=0.1, seed=seed1, N=20, population_options='select N=20', population_division=12,
               N_green=0, N_cost=1, N_convenience=2, N_indifferent=3, supply_factor=1, cost_comparison=p['cost_comparison'], reduction=p['reduction'],
               env_score_setting=p['env_score'], long_term_effect_info=17505, n_info_campaign_environment=p['info_campaign_env'], effect_info_campaign_environment=p["effect_info_campaign_environment"],
               n_info_campaign_smart_meter=p['info_campaign_sm'], effect_info_campaign_smart_meter=p["effect_info_campaign_smart_meter"], init_package=p['init_package'],runtime=(len(green_list)-1)):

    model = GridModel(green_list_year, fossil_list_year, green_list, fossil_list, 
                      semishift_list, shift_list, demand_list, green_prediction, price_flat,
                      alpha, beta, gamma, RTP_policy, switch_period, threshold_shift,
                      threshold_semishift, social_pressure_effect, seed, N, population_options, population_division,
                      N_green, N_cost, N_convenience, N_indifferent, supply_factor, cost_comparison, reduction, env_score_setting,
                      long_term_effect_info, n_info_campaign_environment,effect_info_campaign_environment, 
                      n_info_campaign_smart_meter, effect_info_campaign_smart_meter, init_package)  
  
    # run model for a specified runtime
    model.run_model(runtime)
    # model output
    model_df = model.datacollector.get_model_vars_dataframe().reset_index()
    
    # select the outcomes of interest from the model output
    return  sum(np.array(model_df['energy shortage'])), model_df['energy shortage'], 100-min(np.array(model_df['storage SoC'][20:])), model_df['storage SoC'], sum(np.array(model_df['Total Energy Costs'])), model_df['Non-shifters share'],model_df['Shifters share'], model_df['Full shifters share'],model_df['Non-shifters share'][0],model_df['Shifters share'][0], model_df['Full shifters share'][0]

In [ ]:
from ema_workbench import Model
# instantiate the model
model = Model('demandresponse', function=grid_model)

# specify uncertainties for set 1, and optional uncertainties for set 2 and 3 
model.uncertainties = [IntegerParameter('population_division', 0, 41),
                       RealParameter('supply_factor', 0.8, 1.4)]


# specify levers - in case you are interested in different policy setups

# cost comparison and reduction policy
#model.levers = [IntegerParameter("reduction", 1, 80)]

# environmental awareness campaign
# model.levers = [IntegerParameter("n_info_campaign_environment", 1, 4),
#                 RealParameter("effect_info_campaign_environment", 1.05, 1.20),
#                 IntegerParameter('long_term_effect_info',0,35010] 

# smart meter information campaign
#model.levers = [IntegerParameter("n_info_campaign_smart_meter", 1, 4),
#                 RealParameter("effect_info_campaign_smart_meter", 0.9, 0.975),
#                 IntegerParameter('long_term_effect_info',0,35010]   


# specify outcomes 
model.outcomes = [ScalarOutcome('Total energy shortage'),
                  TimeSeriesOutcome('Energy shortage'),
                  ScalarOutcome('Required storage'),
                  TimeSeriesOutcome('Battery SoC'),
                  ScalarOutcome('Total costs'),
                  TimeSeriesOutcome('No shifter share'),
                  TimeSeriesOutcome('Shifters share'),
                  TimeSeriesOutcome('Full shifters share'),
                  ScalarOutcome('Initial no shifter share'),
                  ScalarOutcome('Initial shifters share'),
                  ScalarOutcome('Initial full shifters share')]


# # override some of the defaults of the model
# model.constants = [Constant('social_pressure_effect', 0),
#                    Constant('alpha', 0.2),
#                    Constant('gamma', 0.2),
#                    Constant('price_flat', 0.345)]

# Experimentation

In [ ]:
# run experiments
ema_logging.log_to_stderr(ema_logging.INFO)

n_scenarios= 20
n_policies = 5

# with SequentialEvaluator(model) as evaluator:
#     experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios,policies=n_policies)

with SequentialEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios)
    
    
# save the results in two ways    

# method 1
# results = experiments, outcomes
# save_results(results, 'Results_EMA/Policies/200s_p0.tar.gz')

# # method 2
# df = experiments
# for outcome in outcomes.keys():
#     df[outcome] = list(outcomes[outcome])

# with gzip.open('Results_EMA/Policies/200s_p0.pkl', "wb") as f:
#     pickled = pickle.dumps(df)
#     optimized_pickle = pickletools.optimize(pickled)
#     f.write(optimized_pickle)

# Analysis of results

### Compare experiments on KPI's  

In [ ]:
# load in the results
experiment0, outcomes0 = load_results('Results_EMA/Policies/200s_p0_fin.tar.gz')
experiment1, outcomes1 = load_results('Results_EMA/Policies/200s_p1_fin.tar.gz')
experiment2a, outcomes2a = load_results('Results_EMA/Policies/200s_p2a_fin.tar.gz')
experiment2b, outcomes2b = load_results('Results_EMA/Policies/200s_p2b_fin.tar.gz')
experiment2c, outcomes2c = load_results('Results_EMA/Policies/200s_p2c_fin.tar.gz')
experiment3, outcomes3 = load_results('Results_EMA/Policies/200s_p3_fin.tar.gz')
experiment4a, outcomes4a = load_results('Results_EMA/Policies/200s_p4a_fin.tar.gz')
experiment4b, outcomes4b = load_results('Results_EMA/Policies/200s_p4b_fin.tar.gz')
experiment4c, outcomes4c = load_results('Results_EMA/Policies/200s_p4c_fin.tar.gz')
experiment4d, outcomes4d = load_results('Results_EMA/Policies/200s_p4d_fin.tar.gz')
experiment4e, outcomes4e = load_results('Results_EMA/Policies/200s_p4e_fin.tar.gz')
experiment5, outcomes5 = load_results('Results_EMA/Policies/200s_p5_fin.tar.gz')
experiment6, outcomes6 = load_results('Results_EMA/Policies/200s_p6_fin.tar.gz')
experiment7, outcomes7 = load_results('Results_EMA/Policies/200s_p7_fin.tar.gz')
experiment8, outcomes8 = load_results('Results_EMA/Policies/200s_p8_fin.tar.gz')
experiment9, outcomes9 = load_results('Results_EMA/Policies/200s_p9_fin.tar.gz')
experiment10, outcomes10 = load_results('Results_EMA/Policies/200s_p10_fin.tar.gz')
experiment11, outcomes11 = load_results('Results_EMA/Policies/200s_p11_fin.tar.gz')
experiment12, outcomes12 = load_results('Results_EMA/Policies/200s_p12_fin.tar.gz')
experiment13, outcomes13 = load_results('Results_EMA/Policies/200s_p13_fin.tar.gz')

In [ ]:
ps = ['p0','p1','p2a','p2b','p2c','p3','p4a','p4b','p4c','p4d','p4e','p5','p6','p7','p8','p9','p10','p11','p12','p13']

outc_0 = []
outc_1 = []
outc_2a = []
outc_2b = []
outc_2c = []
outc_3 = []
outc_4a = []
outc_4b = []
outc_4c = []
outc_4d = []
outc_4e = []
outc_5 = []
outc_6 = []
outc_7 = []
outc_8 = []
outc_9 = []
outc_10 = []
outc_11 = []
outc_12 = []
outc_13 = []

for i in range(200):
    outc_0.append(outcomes0["Total energy shortage"][i])
    outc_1.append(outcomes1["Total energy shortage"][i])
    outc_2a.append(outcomes2a["Total energy shortage"][i])
    outc_2b.append(outcomes2b["Total energy shortage"][i])
    outc_2c.append(outcomes2c["Total energy shortage"][i])
    outc_3.append(outcomes3["Total energy shortage"][i])
    outc_4a.append(outcomes4a["Total energy shortage"][i])
    outc_4b.append(outcomes4b["Total energy shortage"][i])
    outc_4c.append(outcomes4c["Total energy shortage"][i])
    outc_4d.append(outcomes4d["Total energy shortage"][i])
    outc_4e.append(outcomes4e["Total energy shortage"][i])
    outc_5.append(outcomes5["Total energy shortage"][i])
    outc_6.append(outcomes6["Total energy shortage"][i])
    outc_7.append(outcomes7["Total energy shortage"][i])
    outc_8.append(outcomes8["Total energy shortage"][i])
    outc_9.append(outcomes9["Total energy shortage"][i])
    outc_10.append(outcomes10["Total energy shortage"][i])
    outc_11.append(outcomes11["Total energy shortage"][i])
    outc_12.append(outcomes12["Total energy shortage"][i])
    outc_13.append(outcomes13["Total energy shortage"][i])
    
# boxplot 
plt.figure(figsize=(15,8))
sns.boxplot(x = ['p0','p1','p2a','p2b','p2c','p3','p4a','p4b','p4c','p4d',
                 'p4e','p5','p6','p7','p8','p9','p10','p11','p12','p13'], 
            y = [outc_0, outc_1, outc_2a, outc_2b, outc_2c, outc_3, outc_4a, outc_4b,
                 outc_4c, outc_4d, outc_4e, outc_5, outc_6, outc_7, outc_8, outc_9,
                 outc_10, outc_11, outc_12, outc_13], palette = 'Greens')
plt.ylabel('Energy shortage')
plt.xlabel('Policies')
plt.title("Energy shortage per policy")

plt.show()

###  Analyze separate policies

In [ ]:
'''Examples of loading in the results'''

# Base case B   
with gzip.open('Results_EMA/Policies/200s_p1_fin.pkl', 'rb') as f:
    pick = pickle.Unpickler(f)
    df = pick.load()
    
# Specific policy results - sampled over uncertainties
with gzip.open('Results_EMA/Policies/200s_p4b_fin.pkl', 'rb') as f:
    pick = pickle.Unpickler(f)
    dfo = pick.load()
    
# Different setups of smart meter campaign - sampled over uncertainties and levers
with gzip.open('Results_EMA/Policies/info_sm_50s_10p.pkl', 'rb') as f:
    pick = pickle.Unpickler(f)
    dfo1 = pick.load()

In [ ]:
'''Data formatting of results to make them suitable for plotting'''
L = [[20,0,0,0],[0,20,0,0],[0,0,20,0],[0,0,0,20],[10,10,0,0],[10,0,10,0],[10,0,10,0],[0,10,10,0],[0,0,10,10],[10,0,0,10],[0,10,0,10],
                 [5,5,0,10],[5,5,5,5],[5,5,10,0],[5,0,5,10],[0,5,5,10],[10,5,0,5],[5,10,0,5],[0,10,5,5],[10,0,5,5],[10,5,0,5],[10,5,5,0],
                 [0,5,0,15],[15,0,0,5],[0,15,0,5],[5,0,15,0],[0,5,15,0],[0,15,0,5],[5,0,0,15],[5,15,0,0],[15,0,5,0],[15,5,0,0],
                 [18,0,0,2],[0,18,0,2],[2,0,0,18],[0,2,0,18],[1,1,0,18],[18,1,1,0],[0,18,2,0],[2,0,18,0],[9,9,1,1],[1,1,9,9]]

def clean_df(dfo):
    # split population in types
    gr = []
    cc = []
    co = []
    ind = []
    pop = []
    for i in range(len(dfo['population_division'])):
        popu = L[int(dfo['population_division'][i])]
        gr.append(popu[0])
        cc.append(popu[1])
        co.append(popu[2])
        ind.append(popu[3])
        popu = str(popu)

    dfo['population'] = popu
    dfo['Green'] = gr
    dfo['Cost-conscious'] = cc
    dfo['Convenience-oriented'] = co
    dfo['Indifferent'] = ind
    
    # select contract shares at end and change
    bb = []
    cc = []
    dd = []
    for i in range(len(dfo['No shifter share'])):
        bb.append(dfo['No shifter share'][i][-1])
        cc.append(dfo['Shifters share'][i][-1])
        dd.append(dfo['Full shifters share'][i][-1])

    dfo['no_shift_share_final'] = bb
    dfo['shift_share_final'] = cc
    dfo['full_shift_share_final'] = dd
    dfo['No shiftshare change'] =  dfo['no_shift_share_final'] - dfo['Initial no shifter share']
    dfo['Shiftshare change'] =  dfo['shift_share_final'] -dfo['Initial shifters share']
    dfo['Full shiftshare change'] =  dfo['full_shift_share_final'] - dfo['Initial full shifters share']
    
    # make a column with cumulative shortage
    dfo['cumulative shortage'] = dfo['Energy shortage']
    for i in range(len(dfo)):
        dfo['cumulative shortage'][i] = dfo['Energy shortage'][i].cumsum()
        
    return dfo

df = clean_df(df)    
dfo = clean_df(dfo)
dfo1 = clean_df(dfo1)

In [ ]:
'''Feature scoring'''
from ema_workbench.analysis import feature_scoring

#for the influences on contract switching. 
dfo_exp =  dfo.loc[:,['Initial no shifter share', 'supply_factor',
                      'Initial shifters share', 'Initial full shifters share', 'Green',
                      'Cost-conscious', 'Convenience-oriented', 'Indifferent']]

dfo_out = dfo.loc[:,['No shiftshare change', 'Shiftshare change', 'Full shiftshare change']]

# #for the influences on the model outcomes
dfo_exp =  dfo.loc[:,['Initial no shifter share','Initial shifters share', 'Initial full shifters share', 
                      'Green', 'Cost-conscious', 'Convenience-oriented', 'Indifferent',
                      'No shiftshare change', 'Shiftshare change', 'Full shiftshare change']]

dfo_out = dfo.loc[:,['Total energy shortage', 'Required storage', 'Total costs']]

x = dfo_exp
y = dfo_out

plt.figure(figsize=(10,8))
fs = feature_scoring.get_feature_scores_all(x, y)
sns.heatmap(fs, cmap='viridis', annot=True)

plt.show()

In [ ]:
'''Plot energy shortage (or other KPI) for all scenarios in an experiment'''
plt.figure(figsize=(13,8))

for i in range(len(df)):
    plt.plot(dfo['Energy shortage'][i].cumsum())
plt.ylabel('Cumulative energy shortage',fontsize=15)
plt.xlabel('Time step (15 min.)',fontsize=15)
#plt.title('Energy shortage in base case B')
plt.show()

In [ ]:
'''Compare bandwidths of model outcome range for 2 experiment sets'''
plt.figure(figsize=(15,8))

x = np.linspace(0, 35010, 35010)
a = np.array(dfo1['cumulative shortage'])
mean1 = np.average(a, axis=0)
std1 = np.std(a)

a = np.array(df['cumulative shortage'])
mean2 = np.average(a, axis=0)
std2 = np.std(a)

plt.plot(x, mean1, '-',label='Smart Meter information Campaign')
plt.fill_between(x, mean1-std1, mean1+std1, alpha = 0.3, color = "green")
plt.plot(x, mean2, '-',label='Base case B')
plt.fill_between(x, mean2-std2, mean2+std2, alpha = 0.2, color = "red")

plt.ylabel('Cumulative energy shortage',fontsize=15)
plt.xlabel('Time step (15 min.)',fontsize=15)
plt.legend(fontsize=15)

In [ ]:
'''Examples of plots to find out the  effect of different reductions'''
# scatterplot
plt.figure(figsize=(10,8))
sns.scatterplot(data=dfo1, x='Total costs', y='Total energy shortage', hue = 'reduction')
plt.show()

# pairplot
dfo_copy = dfo1
for i in range(len(dfo_copy['reduction'])):
    if dfo_copy['reduction'][i] < 30:
        dfo_copy['reduction'][i] = '< 30'
    elif 30 <  dfo_copy['reduction'][i] < 50:
        dfo_copy['reduction'][i] = '30 - 50'
    else:
        dfo_copy['reduction'][i] = '> 50'

sns.pairplot(dfo_copy, vars=['Total energy shortage', 'Required storage', 'Total costs',
       'No shiftshare change', 'Shiftshare change', 'Full shiftshare change'], hue='reduction')
plt.show()

# density plot of changes in shifting contract
fig, axs = plt.subplots(3)
fig.suptitle('Vertically stacked subplots')
plt.xlabel('Changes in shares of shifting contracts with different reductions')

dfo.groupby(['reduction'])['No shiftshare change'].plot.kde(figsize=(15,10), ax=axs[0])
axs[0].set_title('Change in share of no shifting option')
dfo.groupby(['reduction'])['Shiftshare change'].plot.kde(figsize=(15,10),ax=axs[1])
axs[1].set_title('Change in share of shifting option')
dfo.groupby(['reduction'])['Full shiftshare change'].plot.kde(figsize=(15,10),ax=axs[2])
axs[2].set_title('Change in share of full shifting option')
plt.legend()
plt.show()